***Transfer learning‑based CNN for plant leaf
disease detection***

 **CVPR[B] Group-10**
* MD. ASIFUR RAHMAN - - -[20-43064-1]
* ABRAR IBNE AHSAN - - - [19-41389-3]
* SHINZON SIDDIQUA - - - [20-43671-2]
* H.M.Saifullahil Mazid- [20-44018-2]

In [ ]:
# CVPR[B] Group-10
#MD. ASIFUR RAHMAN - - - [20-43064-1]
#ABRAR IBNE AHSAN -	 - - [19-41389-3]
#SHINZON SIDDIQUA - - - -[20-43671-2]
#H.M.Saifullahil Mazid- -[20-44018-2]


import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import layers, Sequential, losses

In [ ]:
import shutil
import os

#source and destination directories
source_directory = "/kaggle/input/plant-village/PlantVillage"
destination_directory = "/kaggle/output/PlantVillage"

folders_to_copy = ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

#if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)


for folder_name in folders_to_copy:
    source_folder = os.path.join(source_directory, folder_name)
    destination_folder = os.path.join(destination_directory, folder_name)

    try:
        shutil.copytree(source_folder, destination_folder)
        print(f"Folder '{folder_name}' copied successfully.")
    except FileExistsError:
        # if folder already exists
        shutil.copy2(source_folder, destination_folder)
        print(f"Folder '{folder_name}' contents copied successfully.")
    except shutil.Error as e:
        print(f"Error copying folder '{folder_name}': {e}")

In [ ]:
# Initializing some Constants
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [ ]:
# Creating the image dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/output/PlantVillage",
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
)
# class names
class_names = dataset.class_names
print("Class Names:", class_names)

In [ ]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

In [ ]:
len(dataset)

In [ ]:
# Data Visualization
plt.figure(figsize=(12,8))
for image_batch, label_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4,i+1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.title(class_names[label_batch[i].numpy()], fontsize = 7)
        plt.axis('off')

In [ ]:
def get_dataset_partitions(dataset, train_size=0.8, val_size=0.1, test_size=0.1, shuffle=True, shuffle_size=10000):
    # Length of Dataset
    dataset_size = len(dataset)

    # Shuffling
    if shuffle:
        dataset = dataset.shuffle(shuffle_size, seed=12)

    # Splitting
    train_size = int(dataset_size * train_size)
    val_size = int(dataset_size * val_size)

    # Training
    train_dataset = dataset.take(train_size)

    # Validation and Testing
    val_dataset = dataset.skip(train_size).take(val_size)
    test_dataset = dataset.skip(train_size).skip(val_size)

    return train_dataset, val_dataset, test_dataset

In [ ]:
train_dataset, val_dataset, test_dataset = get_dataset_partitions(dataset)

In [ ]:
print(f"Length of Training Dataset: {len(train_dataset)}")
print(f"Length of Validation Dataset: {len(val_dataset)}")
print(f"Length of Testing Dataset: {len(test_dataset)}")
print(f"Is the length of the whole dataset equal to the length of the sum of the splits?",
     (len(dataset) == (len(train_dataset) + len(val_dataset) + len(test_dataset))))

In [ ]:
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Initializing some constants
N_CLASSES = len(class_names)
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

In [ ]:
resize_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.,255)
])

data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(resize_rescale)
model.add(data_augmentation)
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(N_CLASSES, activation='softmax'))

model.build(input_shape=input_shape)
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_dataset
)

In [ ]:
score = model.evaluate(test_dataset)

In [ ]:
history.params

In [ ]:
history.history.keys()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import numpy as np
for images_batch, labels_batch in test_dataset.take(1):

    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])

    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))

        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]]

        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")

        plt.axis("off")

In [ ]:
# CVPR[B] Group-10
#MD. ASIFUR RAHMAN - - - [20-43064-1]
#ABRAR IBNE AHSAN -	 - - [19-41389-3]
#SHINZON SIDDIQUA - - - -[20-43671-2]
#H.M.Saifullahil Mazid- -[20-44018-2]